In [ ]:
from eumap.misc import find_files
from eumap import parallel
import datatable as dt
import geopandas as gpd
import pandas as pd
from pathlib import Path
import seaborn as sns
sns.set_theme()

wd = '/mnt/tupi/WRI/livestock_global_modeling/livestock_census_raw'
ard_census_dir='/mnt/tupi/WRI/livestock_global_modeling/livestock_census_ard'

All processing steps to integrate the census livestock data is implemented following, considering Global Pasture Watch (GPW), FAO (Gridded Livestock of the World v4), Malek et al., 2024 and FAOSTAT.

## GPW data

In [ ]:
CENSUS_DB=f'{wd}/ArealDB_GPW'

adb_tables_path=f'{CENSUS_DB}/tables'
adb_geometries_path=f'{CENSUS_DB}/geometries'

### Reading number of animals

In [ ]:
def _read_csv(csv_file):
    return dt.fread(csv_file).to_pandas()

args = [ (f,) for f in find_files(adb_tables_path, '*.csv') ]
df = pd.concat([ df for df in parallel.job(_read_csv, args, n_jobs=-1) ])
df['ontoMatch'] = df['ontoMatch'].str.lower()
                 
df['animal'] = df['ontoName'].str.lower()
df['level'] = df['gazName'].str.split('.', expand=False).str.len()
df['country'] = df['gazName'].str.split('.', expand=True)[0]
df['year'] = pd.to_numeric(df['year'].astype('string').str.slice(0,4), errors='coerce')
df = df.reset_index(drop=True)

df['country'] = df['country'].replace('United States','United States of America')

### Sanity checks

#### Brazil double check

In [ ]:
df_br = df.query("country == 'Brazil' and level > 2 and gazName == 'Brazil.Goiás.Jataí' and year == 2021")
df_br

## GPW Level 2+

In [ ]:
df_gpw = df.query('year >= 2000 & number_heads >= 0 & level >= 2')

mask = (df_gpw['animal'] == '')
df_gpw.loc[mask,'animal'] = df_gpw[mask]['ontoMatch'].map({
       'овцы и козы': 'sheep and goats',
       'livestock - meat cattle - cows and heifers 1 year and over (no)': 'cattle',
       'live bovine animals': 'cattle',
       'bovine animals, less than 1 year old': 'cattle',
       'livestock - dairy cattle - cows in milk and dry (no)': 'cattle',
       'livestock - meat cattle - calves less than 1 year (no)': 'cattle',
       'bovine animals, less than 1 year old, for slaughter': 'cattle',
       'livestock - dairy cattle - heifers 1 to 2 years (no)': 'cattle',
       'livestock - dairy cattle - heifers over 2 years (no)': 'cattle',
       'bovine animals, less than 1 year old, not for slaughter': 'cattle',
       'bovine animals, 1 to less than 2 years old': 'cattle',
       'bovine animals, 2 years old or over': 'cattle',
       'livestock - meat cattle - all other (no)': 'cattle',
       'livestock - sheep and lambs - breeding ewes 1 year and over - other breeding ewes nec (no)': 'sheep',
       'livestock - sheep and lambs - all other (no)': 'sheep',
       'livestock - sheep and lambs - breeding ewes 1 year and over - merinos (no)': 'sheep'
}).fillna('')

df_gpw[df_gpw['level'] >= 2][['gazID','year']].value_counts().reset_index().groupby('gazID').agg({'year':'count', 'count': 'sum'}).reset_index().sort_values('year', ascending=False)

In [ ]:
import numpy as np
country_stats = df_gpw[np.logical_and(df_gpw['level'] >= 2, df_gpw['gazID'] != '')][['country','year']].value_counts().reset_index().groupby('country').agg({'year':'count', 'count': 'sum'}).reset_index().sort_values('year', ascending=False)
country_stats

#### Geometries

In [ ]:
# pip install pyogrio
def read_geom(geom_dir, country, level):
    try:
        gdf = gpd.read_file(f'{geom_dir}/{country}.gpkg', layer=f'al{level}' )
        return gdf[['gazID', 'gazName','gazClass','geoID','geometry']]
    except:
        try:
            gdf = gpd.read_file(f'{geom_dir}/{country}.gpkg', layer=f'ADM{int(level) - 1}' )
            return gdf[['gazID', 'gazName','gazClass','geoID','geometry']]
        except:
            return gpd.GeoDataFrame([])
    
args = [ (adb_geometries_path, cl.split('#')[0], cl.split('#')[1]) for cl in pd.unique(df_gpw['country']+ '#' + df_gpw['level'].astype(str)) ]
#args[0]
gdf_gpw = pd.concat([ df for df in parallel.job(read_geom, args, n_jobs=40) ])

# Only geometries with livesotck information 
gdf_gpw = gdf_gpw[gdf_gpw['gazName'].isin(df_gpw['gazName'].unique())]
gdf_gpw = gdf_gpw.drop_duplicates(['gazID'], keep='first', ignore_index=True)
gdf_gpw

In [ ]:
gdf_gpw_countries = pd.DataFrame(gdf_gpw['gazName'].str.split('.', expand=True)[0].value_counts()).reset_index().sort_values(0).rename(columns={0:'country'})
gdf_gpw_countries[gdf_gpw_countries['country'].isin(['Mexico', 'Bolivia', 'Colombia', 'Peru', 'Paraguay', 'Chile', 'Ukraine'])]

In [ ]:
## Ad-hoc fixing for names
ngazid_mask = (df_gpw['gazID'] == '')
df_gpw.loc[:,'gazMatch_new'] = df_gpw['gazMatch'].str.title() \
    .str.replace('][','.') \
    .str.replace('United States','United States of America')

df_gpw.loc[ngazid_mask, ['gazMatch_new0','gazMatch_new1','gazMatch_new2']] = df_gpw[ngazid_mask]['gazMatch_new'].str.split('.', expand=True).to_numpy()[:, 0:3]
gdf_gpw.loc[:, ['gazMatch_new0','gazMatch_new1','gazMatch_new2']] = gdf_gpw['gazName'].str.split('.', expand=True).to_numpy()[:, 0:3]

#### googletrans

In [ ]:
#pip install googletrans==3.1.0a0
from googletrans import Translator
import numpy as np

def translate(x):
    translator = Translator()
    text = translator.translate(x, dest='en').text
    #text = text.lower().replace('city', '') \
    #    .replace('districts', '') \
    #    .replace('municipal','') \
    #    .replace('district','') \
    #    .replace('region','') \
    #    .replace('of the ','') \
    #    .replace("urban","") \
    #    .replace("autonomous","")
    return text

mask = np.logical_and(ngazid_mask, df_gpw['gazMatch_new0'] == 'Russian Federation')
#ru_map = {}
#for t in df_gpw[mask]['gazMatch_new2'].unique():
#    ru_map[t] = translate(t)
#ru_map

In [ ]:
ru_map = {
	'Городской Округ Вахрушев': 'Vakhrushev', 
	'Зато Фокино': "Fokino", 
	'Александро-Невский Муниципальный Район': 'Alexandro-Nevsky', 
	'Федоровский Муниципальный Район': 'Fedorovsky', 
	'Калининский Муниципальный Район': 'Kalininsky', 
	'Камышлинский Муниципальный Район': 'Kamyshlinsky', 
	'Клявлинский Муниципальный Район': 'Klyavlinsky', 
	'Краснинский Муниципальный Район': 'Krasninsky', 
	'Красноярский Муниципальный Район': 'Krasnoyarsk', 
	'Ленский Муниципальный Район': 'Lensky', 
	'Северо-Курильский Городской Округ': 'North Kuril', 
	'Островский Муниципальный Район': 'Ostrovsky', 
	'Починковский Муниципальный Район': 'Pochinkovsky',
	'Похвистнево': 'Pokhvistnevo', 
	'Городские Округа Приморского Края': 'Primorsky Krai', 
	'Приволжский Муниципальный Район': 'Privolzhsky', 
	'Городские Округа Рязанской Области': 'Ryazan Region', 
	'Городские Округа Республики Саха (Якутии)': 'Sakha (Yakutia)', 
	'Город Саратов': 'Saratov', 
	'Южно-Курильский Городской Округ': 'South Kuril', 
	'Город Великие Луки': 'Velikie Luki', 
	'Волжский Муниципальный Район': 'Volzhsky', 
	'Воскресенский Муниципальный Район': 'Voskresensky', 
	'Вяземский Муниципальный Район': 'Vyazemsky', 
	'Город Южно-Сахалинск': 'Yuzhno-Sakhalinsk', 
	'Янтарный': 'Amber', 
	'Город Новосибирск': 'Novosibirsk', 
	'Абатский Муниципальный Район': 'Abatsky', 
	'Город Абаза': 'Abaza', 
	'Город Ачинск': 'Achinsk', 
	'Городские Округа Республики Адыгеи (Адыгеи)': 'Adygea', 
	'Сельские Муниципальные Районы Республики Адыгеи': 'Adygea', 
	'Город Адыгейск': 'Adygeisk', 
	'Город Агидель': 'Agidel', 
	'Алексеевский Муниципальный Район И Город Алексеевка': 'Alekseevsky and Alekseevka', 
	'Алексеевский Муниципальный Район': 'Alekseevsky', 
	'Алексеевский': 'Alekseevsky', 
	'Населенные Пункты Входящие В Состав Городского Округа  Город Алексин': 'Aleksin', 
	'Сельские Поселения Алексинского Муниципального Района': 'Aleksinsky', 
	'Александровский Муниципальный Район': 'Alexandrovsky', 
	'Сельские Поселения Александровского Муниципального Района  (До 2021 Года)': 'Alexandrovsky', 
	'Алтайский Муниципальный Район': 'Altai', 
	'Городской Округ Алушта': 'Alushta', 
	'Городской Округ Анадырь': 'Anadyr', 
	'Сельские Поселения Андроповского Муниципального Района (До 2021 Года)': 'Andropov', 
	'Сельские Поселения Апанасенковского Муниципального Района (До 2021 Года)': 'Apanasenkovsky', 
	'Ардатовский Муниципальный Район': 'Ardatovsky', 
	'Архаринский Муниципальный Район': 'Arkharinsky', 
	'Городской Округ Армянск': 'Armyansk', 
	'Сельские Поселения Арсеньевского Муниципального Района': 'Arsenyevsky', 
	'Сельские Поселения Арзгирского Муниципального Района (До 2021 Года)': 'Arzgir', 
	'Багратионовский': 'Bagrationovsky', 
	'Бахчисарайский Муниципальный Район': 'Bakhchisaraysky', 
	'Баксан': 'Baksan', 
	'Baltiysk': 'Baltic', 
	'Балтийский Муниципальный Район': 'Baltic', 
	'Городские Округа Республики Башкортостан': 'Bashkortostan', 
	'Башмаковский Муниципальный Район': 'Bashmakovsky', 
	'Бековский Муниципальный Район': 'Bekovsky', 
	'Сельские Поселения Белевского Муниципального Района': 'Belevsky', 
	'Город Белгород': 'Belgorod', 
	'Белгородский Муниципальный Район': 'Belgorodsky', 
	'Белинский Муниципальный Район': 'Belinsky', 
	'Город Белогорск': 'Belogorsk', 
	'Белогорский Муниципальный Район': 'Belogorsky', 
	'Беловский Муниципальный Район': 'Belovsky', 
	'Березовский Муниципальный Район': 'Berezovsky', 
	'Бессоновский Муниципальный Район': 'Bessonovsky', 
	'Город Биробиджан': 'Birobidzhan', 
	'Городские Поселения Благодарненского Муниципального Района (До 2018 Года)': 'Blagodarnensky', 
	'Сельские Поселения Благодарненского Муниципального Района (До 2018 Года)': 'Blagodarnensky', 
	'Город Благовещенск': 'Blagoveshchensk', 
	'Благовещенский Муниципальный Район': 'Blagoveshchensky', 
	'Бодайбинский Муниципальный Район': 'Bodaybinsky', 
	'Городские Поселения Богородицкого  Муниципального Района': 'Bogoroditsky', 
	'Сельские Поселения Богородицкого Муниципального Района': 'Bogoroditsky', 
	'Богородский Муниципальный Район': 'Bogorodsky', 
	'Богородский': 'Bogorodsky', 
	'Борисовский Муниципальный Район': 'Borisovsky', 
	'Бронницы': 'Bronnitsy', 
	'Городские Округа Брянской Области': 'Bryansk', 
	'Сельские Поселения Буденновского Муниципального Района (До 2021 Года)': 'Budennovsky', 
	'Бурейский Муниципальный Район': 'Bureysky', 
	'Зато Северск': 'But Seversk', 
	'Городские Округа Челябинской Области': 'Chelyabinsk', 
	'Городские Округа Челябинской Области': 'Chelyabinsk', 
	'Город Черногорск': 'Chernogorsk', 
	'Черноморский Муниципальный Район': 'Chernomorsky', 
	'Сельские Поселения Чернского Муниципального Района': 'Chernsky', 
	'Чернянский Муниципальный Район': 'Chernyansky', 
	'Город Чкаловск': 'Chkalovsk', 
	'Городские Округа Республики Крым': 'Crimea', 
	'Муниципальные Районы Республики Крым': 'Crimea', 
	'Городские Округа Республики Дагестан': 'Dagestan', 
	'Дубенский Муниципальный Район': 'Dubensky', 
	'Сельские Поселения Дубенского Муниципального Района': 'Dubensky', 
	'Дубна': 'Dubna', 
	'Город Дзержинск': 'Dzerzhinsk', 
	'Дзержинский Муниципальный Район': 'Dzerzhinsky', 
	'Джанкойский Муниципальный Район': 'Dzhankoysky', 
	'Джейрахский Муниципальный Район': 'Dzheyrakhsky', 
	'Населенные Пункты Входящие В Состав Городского Округа  Город Ефремов': 'Efremov', 
	'Городской Округ Эгвекинот': 'Egvekinot', 
	'Ермекеевский Муниципальный Район': 'Ermekeyevsky', 
	'Эвенкийский Муниципальный Район': 'Evenki', 
	'Городской Округ Феодосия': 'Feodosia', 
	'Город Фокино': 'Fokino', 
	'Георгиевский  Городской  Округ (До 2018 Года Г Георгиевск)': 'Georgievsky', 
	'Сельские Поселения Георгиевского Муниципального Района (До 2018 Года)': 'Georgievsky', 
	'Гиагинский Муниципальный Район': 'Giaginsky', 
	'Го Воротынский': 'Go Vorotynsky', 
	'Городищенский Муниципальный Район': 'Gorodishche', 
	'Сельские Поселения Грачевского Муниципального Района (До 2021 Года)': 'Grachevsky', 
	'Грайворонский Муниципальный Район': 'Grayvoronsky', 
	'Грайворонский': 'Grayvoronsky', 
	'Город Губкин И Губкинский Муниципальный Район': 'Gubkin and Gubkinsky', 
	'Губкинский': 'Gubkinsky', 
	'Гурьевский': 'Guryevsky', 
	'Гусевский': 'Gusevsky', 
	'Гвардейский': 'Gvardeysky', 
	'Городские Округа С Внутригородским Делением': 'Intra Divisions', 
	'Городские Поселения Ипатовского Муниципального Района (До 2018 Года)': 'Ipatovsky', 
	'Сельские Поселения Ипатовского Муниципального Района (До 2018 Года)': 'Ipatovsky', 
	'Городские Округа Иркутской Области/': 'Irkutsk', 
	'Иссинский Муниципальный Район': 'Issinsky', 
	'Ивановский Муниципальный Район': 'Ivanovsky', 
	'Ивнянский Муниципальный Район': 'Ivnyansky', 
	'Сельские Поселения Изобильненского Муниципального Района (До 2018 Года)': 'Izobilnensky', 
	'Городские Округа Еврейской Автономной Области': 'Jewish Autonomous', 
	'Калининград': 'Kaliningrad', 
	'Городские Округа Республики Калмыкии': 'Kalmykia', 
	'Калтанский': 'Kaltansky', 
	'Каменский Муниципальный Район': 'Kamensky', 
	'Сельские Поселения Каменского Муниципального Района': 'Kamensky', 
	'Камешкирский Муниципальный Район': 'Kameshkir', 
	'Город Карабулак': 'Karabulak', 
	'Городские Округа Республики Карелия': 'Karelia', 
	'Муниципальные Районы Республики Карелия': 'Karelia', 
	'Кайбицкий Муниципальный Район': 'Kaybitsky', 
	'Город Казань': 'Kazan', 
	'Город Кедровый': 'Kedrovy', 
	'Городской Округ Керчь': 'Kerch', 
	'Городские Округа Хабаровского Края/': 'Khabarovsk', 
	'Городские Округа Ханты-Мансийского Автономного Округа - Югры': 'Khanty-Mansiysk Autonomous Okrug - Ugra', 
	'Районные Округа Ханты-Мансийского Автономного Округа - Югры': 'Khanty-Mansiysk Autonomous Okrug - Ugra', 
	'Город Хасавюрт': 'Khasavyurt', 
	'Городские Поселения Кимовского Муниципального Района': 'Kimovsky', 
	'Сельские Поселения Кимовского Муниципального Района': 'Kimovsky', 
	'Городские Поселения Киреевского Муниципального Района': 'Kireyevsky', 
	'Городские Поселения Кировского Муниципального Района (До 2018 Года)': 'Kirov', 
	'Сельские Поселения Кировского Муниципального Района (До 2018 Года)': 'Kirov', 
	'Кировский Муниципальный Район': 'Kirovsky', 
	'Город-Курорт Кисловодск': 'Kislovodsk', 
	'Сельские Поселения Кочубеевского Муниципального Района (До 2021 Года)': 'Kochubeevsky', 
	'Колышлейский Муниципальный Район': 'Kolyshleysky', 
	'Городские Округа Республики Коми': 'Komi', 
	'Муниципальные Районы Республики Коми': 'Komi', 
	'Комсомольский Муниципальный Район': 'Komsomolsky', 
	'Константиновский Муниципальный Район': 'Konstantinovsky', 
	'Корочанский Муниципальный Район': 'Korochansky', 
	'Королев': 'Korolev', 
	'Кошехабльский Муниципальный Район': 'Koshekhablsky', 
	'Городские Округа Костромской Области': 'Kostroma Region', 
	'Красненский Муниципальный Район': 'Krasnensky', 
	'Красноармейский Муниципальный Район': 'Krasnoarmeysky', 
	'Краснобродский': 'Krasnobrodsky', 
	'Сельские Поселения Красногвардейского Муниципального Района (До 2021 Года)': 'Krasnogvardeisky', 
	'Красногвардейский Муниципальный Район': 'Krasnogvardeysky', 
	'Краснокамский Муниципальный Район': 'Krasnokamsk', 
	'Городской Округ Красноперекопск': 'Krasnoperekopsk', 
	'Красноперекопский Муниципальный Район': 'Krasnoperekopsky', 
	'Краснояружский Муниципальный Район': 'Krasnoyaruzhsky', 
	'Краснознаменский': 'Krasnoznamensky', 
	'Куйбышевский Муниципальный Район': 'Kuibyshevsky', 
	'Кумторкалинский Муниципальный Район': 'Kumtorkalinsky', 
	'Курчалоевский Муниципальный Район': 'Kurchaloyevsky', 
	'Сельские Поселения Куркинского Муниципального Района': 'Kurkinsky', 
	'Сельские Поселения Курского Муниципального Района (До 2021 Года)': 'Kursk', 
	'Город Кузнецк': 'Kuznetsk', 
	'Кузнецкий Муниципальный Район': 'Kuznetsk', 
	'Ладушкинский': 'Ladushkinsky', 
	'Ленинск-Кузнецкий': 'Leninsk-Kuznetsky', 
	'Ленинский Муниципальный Район': 'Leninsky', 
	'Лескенский Муниципальный Район': 'Leskensky', 
	'Город Лесосибирск': 'Lesosibirsk', 
	'Сельские Поселения Левокумского Муниципального Района (До 2021 Года)': 'Levokum', 
	'Ликино-Дулево': 'Likino-Dulevo', 
	'Лопатинский Муниципальный Район': 'Lopatinsky', 
	'Лунинский Муниципальный Район': 'Luninsky', 
	'Магдагачинский Муниципальный Район': 'Magdagachinsky', 
	'Город Малгобек': 'Malgobek', 
	'Малосердобинский Муниципальный Район': 'Maloserdobinsky', 
	'Мамоновский': 'Mamonovsky', 
	'Мантуровский Муниципальный Район': 'Manturovo', 
	'Городские Округа Республики Марий Эл': 'Mari El', 
	'Город Майкоп': 'Maykop', 
	'Майкопский Муниципальный Район': 'Maykopsky', 
	'Мазановский Муниципальный Район': 'Mazanovsky', 
	'Мелекесский Муниципальный Район': 'Melekessky', 
	'Михайловский Муниципальный Район': 'Mikhaylovsky', 
	'Сельские Поселения Минераловодского Муниципального Района (До 2017 Года)': 'Mineralovodsk', 
	'Мишкинский Муниципальный Район': 'Mishkinsky', 
	'Мокшанский Муниципальный Район': 'Mokshansky', 
	'Городские Округа Республики Мордовии/': 'Mordovia', 
	'Муниципальные Образования Республики Мордовии': 'Mordovia', 
	'Муниципальный Район': 'Municipal District', 
	'Нальчик': 'Nalchik', 
	'Наровчатский Муниципальный Район': 'Narovchatsky', 
	'Город Нарьян-Мар': 'Naryan-Mar', 
	'Навашинский': 'Navashinsky', 
	'Город Назарово': 'Nazarovo', 
	'Город Назрань': 'Nazran', 
	'Сельские Поселения Нефтекумского Муниципального Района (До 2018 Года)': 'Neftekumsky', 
	'Нестеровский': 'Nesterovsky', 
	'Неверкинский Муниципальный Район': 'Neverkinsky', 
	'Никольский Муниципальный Район': 'Nikolsky', 
	'Нижнегорский Муниципальный Район': 'Nizhnegorsky', 
	'Нижнеломовский Муниципальный Район': 'Nizhnelomovsky', 
	'Город Норильск': 'Norilsk', 
	'Городские Округа Республики Северной Осетии-Алании': 'North Ossetia-Alania', 
	'Северный Муниципальный Район': 'Northern', 
	'Городские Поселения Новоалександровского Муниципального Района (До 2018 Года)': 'Novoaleksandrovsky', 
	'Сельские Поселения Новоалександровского Муниципального Района (До 2018 Года)': 'Novoaleksandrovsky', 
	'Городские Поселения Новомосковского Муниципального Района': 'Novomoskovsk', 
	'Новооскольский Муниципальный Район': 'Novooskolsky', 
	'Новооскольский': 'Novooskolsky', 
	'Сельские Поселения Новоселицкого Муниципального Района (До 2021 Года)': 'Novoselitsky', 
	'Городской Округ Новозыбковский С 2020 Года': 'Novozybkovsky', 
	'Нурлатский Муниципальный Район': 'Nurlatsky', 
	'Сельские Поселения Одоевского Муниципального Района': 'Odoevsky', 
	'Октябрьский Муниципальный Район': 'Oktyabrsky', 
	'Ольгинский Муниципальный Район': 'Olginsky', 
	'Город Омск': 'Omsk', 
	'Город Орск': 'Orsk', 
	'Орловский Муниципальный Район': 'Oryol', 
	'Озерский': 'Ozersky', 
	'Пачелмский Муниципальный Район': 'Pachelma', 
	'Партизанский Муниципальный Район': 'Partizansky', 
	'Павловский Муниципальный Район': 'Pavlovsky', 
	'Город Пенза': 'Penza', 
	'Пензенский Муниципальный Район (Включая Кондольский С 14-06-2006)': 'Penza', 
	'Перевозский': 'Perevozsky', 
	'Пермский Край': 'Perm Region', 
	'Perm': 'Perm', 
	'Городской Округ Город Пермь': 'Perm', 
	'Город Первомайск': 'Pervomaisk', 
	'Первомайский Муниципальный Район': 'Pervomaysky', 
	'Петропавловск-Камчатский': 'Petropavlovsk-Kamchatsky', 
	'Городские Поселения Петровского Муниципального Района (До 2018 Года)': 'Petrovsky', 
	'Петровский Муниципальный Район': 'Petrovsky', 
	'Сельские Поселения Петровского Муниципального Района (До 2018 Года)': 'Petrovsky', 
	'Городской Округ Певек': 'Pevek', 
	'Сельские Поселения Предгорного Муниципального Района (До 2021 Года)': 'Piedmont', 
	'Пионерский': 'Pionersky', 
	'Сельские Поселения Плавского Муниципального Района': 'Plavsky', 
	'Полесский': 'Polessky', 
	'Полысаевский': 'Polysaevsky', 
	'Правдинский': 'Pravdinsky', 
	'Рабочий Поселок Прогресс': 'Progress', 
	'Прохоровский Муниципальный Район': 'Prokhorovsky', 
	'Городской Округ Провиденский': 'Providensky', 
	'Рабочий Поселок Кольцово': 'Rabochiy Poselok Koltsovo', 
	'Город Райчихинск': 'Raichikhinsk', 
	'Ракитянский Муниципальный Район': 'Rakityansky', 
	'Раздольненский Муниципальный Район': 'Razdolnensky', 
	'Ромненский Муниципальный Район': 'Romnensky', 
	'Ровеньский Муниципальный Район': 'Rovensky', 
	'Сакский Муниципальный Район': 'Saksky', 
	'Санчурский': 'Sanchursky', 
	'Селемджинский Муниципальный Район': 'Selemdzhinsky', 
	'Город Сельцо': 'Seltso', 
	'Семеновский': 'Semenovsky', 
	'Сердобский Муниципальный Район': 'Serdobsky', 
	'Серноводский Муниципальный Район': 'Sernovodsky', 
	'Серышевский Муниципальный Район': 'Seryshevsky', 
	'Город Шахунья': 'Shakhunya', 
	'Шаройский Муниципальный Район': 'Sharoysky', 
	'Город Шарыпово': 'Sharypovo', 
	'Городские Поселения Щекинского Муниципального Района': 'Shchekinsky', 
	'Сельские Поселения Щекинского Муниципального Района': 'Shchekinsky', 
	'Шебекинский Муниципальный Район И Город Шебекино': 'Shebekinsky and Shebekino', 
	'Шебекинский': 'Shebekinsky', 
	'Шемышейский Муниципальный Район': 'Shemyshey', 
	'Город Шимановск': 'Shimanovsk', 
	'Шимановский Муниципальный Район': 'Shimanovsky', 
	'Шовгеновский Муниципальный Район': 'Shovgenovsky', 
	'Городские Поселения Шпаковского Муниципального Района (До 2021 Года)': 'Shpakovsky', 
	'Сельские Поселения Шпаковского Муниципального Района (До 2021 Года)': 'Shpakovsky', 
	'Городской Округ Симферополь': 'Simferopol', 
	'Симферопольский Муниципальный Район': 'Simferopolsky', 
	'Сковородинский Муниципальный Район': 'Skovorodinsky', 
	'Славский': 'Slavsky', 
	'Город Слободской': 'Slobodskaya', 
	'Сокольский Муниципальный Район': 'Sokolsky', 
	'Сокольский': 'Sokolsky', 
	'Соль-Илецкий': 'Sol-Iletsky', 
	'Сорочинский (До 2016 Г - Го Город Сорочинск)': 'Sorochinsky', 
	'Город Сорск': 'Sorsk', 
	'Сосновоборский Муниципальный Район': 'Sosnovoborsky', 
	'Сосновский Муниципальный Район': 'Sosnovsky', 
	'Советский Муниципальный Район': 'Sovetsky', 
	'Городские Поселения Советского Муниципального Района (До 2018 Года)': 'Soviet', 
	'Сельские Поселения Советского Муниципального Района (До 2018 Года)': 'Soviet', 
	'Спасский Муниципальный Район': 'Spassky', 
	'Город Стародуб': 'Starodub', 
	'Город Старый Оскол И Старооскольский Муниципальный Район': 'Stary Oskol and Stary Oskolsky', 
	'Старооскольский': 'Stary Oskolsky', 
	'Городские Округа Ставропольского Края': 'Stavropol', 
	'Сельские Поселения Степновского Муниципального Района (До 2021 Года)': 'Stepnovsky', 
	'Город Стерлитамак': 'Sterlitamak', 
	'Пригородный Муниципальный Район': 'Suburban', 
	'Городской Округ Судак': 'Sudak', 
	'Город Сунжа (С 2017Года)': 'Sunzha', 
	'Городские Поселения Суворовского Муниципального Района': 'Suvorov', 
	'Сельские Поселения Суворовского Муниципального Района': 'Suvorovsky', 
	'Городские Округа Свердловской Области': 'Sverdlovsk', 
	'Светлогорский': 'Svetlogorsky',
	'Светловский': 'Svetlovsky', 
	'Город Свирск': 'Svirsk', 
	'Свободненский Муниципальный Район': 'Svobodnensky', 
	'Город Свободный': 'Svobodny', 
	'Сыктывкар': 'Syktyvkar', 
	'Тайгинский': 'Taiginsky', 
	'Таймырский Долгано-Ненецкий Муниципальный Район': 'Taimyr Dolgano-Nenets', 
	'Тахтамукайский Муниципальный Район': 'Takhtamukaysky', 
	'Тамалинский Муниципальный Район': 'Tamalinsky', 
	'Тамбовский Муниципальный Район': 'Tambovsky', 
	'Сельские Поселения Тепло-Огаревского Муниципального Района': 'Teply-Ogarevsky', 
	'Терский Муниципальный Район': 'Tersky', 
	'Теучежский Муниципальный Район': 'Teuchezhsky', 
	'Городские Округа Курской Области': 'the Kursk Region', 
	'Троицкий Городской Округ': 'Troitsk', 
	'Сельские Поселения Труновского Муниципального Района (До 2021 Года)': 'Trunovsky', 
	'Целинный Муниципальный Район': 'Tselinny', 
	'Цивильский Муниципальный Район': 'Tsivilsky', 
	'Городские Округа Тульской Области': 'Tula', 
	'Районные Округа Тульской Области': 'Tula', 
	'Сельские Поселения Туркменского Муниципального Района (До 2021 Года)': 'Turkmen', 
	'Город Тында': 'Tynda', 
	'Тындинский Муниципальный Район': 'Tyndinsky', 
	'Городские Округа Тюменской Области': 'Tyumen', 
	'Город Уфа': 'Ufa', 
	'Городские Округа Ульяновской Области': 'Ulyanovsk', 
	'Городской Округ С Внутригородским Делением': 'Urban District With Intra Division', 
	'Усольский Муниципальный Район': 'Usolsky', 
	'Сельские Поселения Узловского Муниципального Района': 'Uzlovsky', 
	'Вадинский Муниципальный Район': 'Vadinsky', 
	'Город Валуйки И Валуйский Муниципальный Район': 'Valuyki and Valuysky', 
	'Валуйский': 'Valuysky', 
	'Вейделевский Муниципальный Район': 'Veidelevsky', 
	'Городские Поселения Веневского Муниципального Района': 'Venevsky', 
	'Сельские Поселения Веневского Муниципального Района': 'Venevsky', 
	'Верхнеуральский Муниципальный Район': 'Verkhneuralsky', 
	'Верхний Уфалейский Городской Округ': 'Verkhniy Ufaleisky', 
	'Володарский Муниципальный Район': 'Volodarsky', 
	'Волоконовский Муниципальный Район': 'Volokonovsky', 
	'Воловский Муниципальный Район': 'Volovsky', 
	'Воркута': 'Vorkuta', 
	'Муниципальный Район Вуктыл (До 1 Января 2017 Года)': 'Vuktyl', 
	'Яковлевский Муниципальный Район': 'Yakovlevsky', 
	'Яковлевский': 'Yakovlevsky', 
	'Городской Округ Ялта': 'Yalta', 
	'Городские Округа Ямало-Ненецкого Автономного Округа': 'Yamalo-Nenets Autonomous Okrug', 
	'Районные Округа Ямало-Ненецкого Автономного Округа': 'Yamalo-Nenets Autonomous Okrug', 
	'Сельские Поселения Ясногорского Муниципального Района': 'Yasnogorsky', 
	'Ейский Муниципальный Район': 'Yeisk', 
	'Городской Округ Евпатория': 'Yevpatoria', 
	'Город Южно-Сухокумск': 'Yuzhno-Sukhokumsk', 
	'Сельские Поселения Заокского Муниципального Района': 'Zaoksky', 
	'Заполярный Район': 'Zapolyarny', 
	'Зато Город Заречный': 'Zarechny', 
	'Завитинский Муниципальный Район': 'Zavitinsky', 
	'Заводоуковский Городской Округ': 'Zavodoukovsky', 
	'Зеленоградский Муниципальный Район': 'Zelenograd', 
	'Зеленоградский': 'Zelenogradsky', 
	'Земетчинский Муниципальный Район': 'Zemetchinsky', 
	'Город Зея': 'Zeya', 
	'Зейский Муниципальный Район': 'Zeysky', 
	'Жуковский Муниципальный Район': 'Zhukovsky', 
	'Город Знаменск': 'Znamensk', 
	'Сельские Поселения Бабаевского Муниципального Района': 'Babaevsky',
	'Сельские Поселения Бабушкинского Муниципального Района': 'Babushkinsky',
	'Сельские Поселения Белозерского Муниципального Района': 'Belozersky',
	'Сельские Поселения Вашкинского Муниципального Района': 'Vashkinsky',
	'Городские Поселения Великоустюгского Муниципального Рай': 'Veliky Ustyug',
	'Сельские Поселения Великоустюгского Муниципального Райо': 'Veliky Ustyug',
	'Сельские Поселения Верховажского Муниципального Района': 'Verkhovazhsky',
	'Городские Поселения Вожегодского Муниципального Района': 'Vozhegodsky',
	'Сельские Поселения Вожегодского Муниципального Района': 'Vozhegodsky',
	'Сельские Поселения Вологодского Муниципального Района': 'the Vologda',
	'Сельские Поселения Вытегорского Муниципального Района': 'Vytegorsky',
	'Городские Поселения Грязовецкого Муниципального Района': 'Gryazovets',
	'Сельские Поселения Грязовецкого Муниципального Района': 'Gryazovets',
	'Городские Поселения Кадуйского Муниципального Района': 'Kaduysky',
	'Сельские Поселения Кадуйского Муниципального Района': 'Kaduysky',
	'Городские Поселения Кирилловского Муниципального Района': 'Kirillovsky',
	'Сельские Поселения Кирилловского Муниципального Района': 'Kirillovsky',
	'Сельские Поселения Кичменгско-Городецкого Муниципального Района': 'Kichmengsko-Gorodetsky',
	'Сельские Поселения Междуреченского Муниципального Район': 'Mezhdurechensky',
	'Городские Поселения Никольского Муниципального Района': 'Nikolsky',
	'Сельские Поселения Никольского Муниципального Района': 'Nikolsky',
	'Сельские Поселения Нюксенского Муниципального Района': 'Nyuksen',
	'Сельские Поселения Сокольского Муниципального Района': 'Sokolsky',
	'Сельские Поселения Сямженского Муниципального Района': 'Syamzhensky',
	'Сельские Поселения Тарногского Муниципального Района': 'Tarnog',
	'Сельские Поселения Тотемского Муниципального Района': 'Totem',
	'Сельские Поселения Усть-Кубинского Муниципального Район': 'Ust-Kubinsky',
	'Городские Поселения Устюженского Муниципального Района': 'Ustyuzhensky',
	'Сельские Поселения Устюженского Муниципального Района': 'Ustyuzhensky',
	'Городские Поселения Харовского Муниципального Района': 'Kharovsky',
	'Сельские Поселения Харовского Муниципального Района': 'Kharovsky',
	'Сельские Поселения Чагодощенского Муниципального Района': 'Chagodoshchensky',
	'Сельские Поселения Череповецкого Муниципального Района': 'Cherepovets',
	'Городские Поселения Шекснинского Муниципального Района': 'Sheksninsky',
	'Сельские Поселения Шекснинского Муниципального Района': 'Sheksninsky',
	'Город Воронеж': 'Voronezh',
	'Город Нововоронеж': 'Novovoronezh',
	'Даниловский Муниципальный Район': 'Danilovsky',
	'Рыбинский Муниципальный Район': 'Rybinsk',
	'Город Переславль-Залесский': 'Pereslavl-Zalessky',
	'Знаменский Муниципальный Район': 'Znamensky',
	'Тере-Хольский Муниципальный Район': 'Tere-Khol',
	'Город Ак-Довурак': 'Ak-Dovurak',
	'Юргинский Муниципальный Район': 'Yurginsky',
	'Городские Округа Тюменской Области': 'Tyumen',
	'Белоярский Муниципальный Район': 'Beloyarsky',
	'Кондинский Муниципальный Район': 'Kondinsky',
	'Нефтеюганский Муниципальный Район': 'Nefteyugansk',
	'Сургутский Муниципальный Район': 'Surgut',
	'Ханты-Мансийский Муниципальный Район': 'Khanty-Mansiysk',
	'Городские Округа Ханты-Мансийского Автономного Округа - Югры': 'Khanty-Mansiysk Autonomous Okrug - Ugra',
	'Красноселькупский Муниципальный Район': 'Krasnoselkup',
	'Надымский Муниципальный Район': 'Nadym',
	'Приуральский Муниципальный Район': 'Priuralsky',
	'Пуровский Муниципальный Район': 'Purovsky',
	'Тазовский Муниципальный Район': 'Tazovsky',
	'Шурышкарский Муниципальный Район': 'Shuryshkarsky',
	'Ямальский Муниципальный Район': 'Yamal',
	'Городские Округа Ямало-Ненецкого Автономного Округа/': 'Yamalo-Nenets Autonomous Okrug',
	'Красногорский Муниципальный Район': 'Krasnogorsk',
	'Город Ижевск': 'Izhevsk',
	'Николаевский Муниципальный Район': 'Nikolaevsky',
	'Ульяновский Муниципальный Район': 'Ulyanovsk',
	'Город Новоульяновск': 'Novoulyanovsk',
	'Городские Округа Владимирской Области': 'Vladimir',
	'Дубовский Муниципальный Район': 'Dubovsky',
	'Руднянский Муниципальный Район': 'Rudnyansky',
	'Город-Герой Волгоград': 'Hero Volgograd',
	'Город Камышин': 'Kamyshin',
	'Сельские Поселения Воловского Муниципального Района': 'Volovsky',
	'Сельские Поселения Ефремовского Муниципального Района': 'Efremov',
	'Сельские Поселения Киреевского Муниципального Района': 'Kireyevsky',
	'Сельские Поселения Ленинского Муниципального Района': 'Leninsky',
	'Сельские Поселения Новомосковского Муниципального Района': 'Novomoskovsk',
    'Эвенкийский  Муниципальный Район': 'Evenki',
    'Таймырский  Долгано-Ненецкий Муниципальный Район': 'Taimyr Dolgano-Nenets',
    'Балтийский':'Baltic'
}

In [ ]:
df_gpw.loc[mask,'gazMatch_new2'] = df_gpw[mask]['gazMatch_new2'].replace(ru_map)

#### textdistance

In [ ]:
#import numpy as np
#mask = np.logical_and(ngazid_mask, df_gpw['gazMatch_new0'] != 'Russian Federation')
#gazMatch_df = df_gpw[mask][['gazMatch_new0','gazMatch_new1','gazMatch_new2']]
#chuncks = np.array_split(gazMatch_df,96)

In [ ]:
#args = [ (c, gdf_gpw) for c in chuncks ]

In [ ]:
#from textdistance import levenshtein

#def text_match(chunck, gdf_gpw):
#    result = []
#    for country, coun_df in chunck.groupby('gazMatch_new0'):
#        for gm2 in coun_df['gazMatch_new2'].unique():
#            coun_gpw = gdf_gpw[gdf_gpw['gazMatch_new0'] == country]
#            #print(coun_gpw.shape, country, gm2)
#            r = coun_gpw['gazMatch_new2'].apply(lambda x: levenshtein.distance(x,  gm2))
#            gpw_gm2 = coun_gpw[r == r.min()]['gazMatch_new2'].iloc[0]
#            result.append({
#                'gm2': gm2,
#                'gm2_match': gpw_gm2,
#                'min':r.min()
#            })
#            #return gpw_gm2
#    return pd.DataFrame(result)

#r = gdf_gpw['gazMatch_new2'].apply(lambda x: levenshtein.distance(x,  'Pereslavl-Zalessky'))
#gdf_gpw[r == r.min()]
#dict_gazmatch_new2 = [ d for d in parallel.job(text_match, args, n_jobs=40) ]
#gazmatch_new2 = pd.concat([ df for df in parallel.job(text_match, args, n_jobs=96) ])
#gazmatch_new2

In [ ]:
#gm2_remap = gazmatch_new2.loc[gazmatch_new2['min'] <= 2,['gm2', 'gm2_match']].groupby(['gm2', 'gm2_match']).count().reset_index().set_index('gm2').to_dict()['gm2_match']
#del gm2_remap['Araporã']
#gm2_remap

In [ ]:
#df_gpw.loc[mask, 'gazMatch_new2'] = df_gpw[mask]['gazMatch_new2'].replace(gm2_remap)
#df_gpw[df_gpw['gazMatch_new2'] == 'Unión']

### Fixing names

In [ ]:
gazid_lookup = gdf_gpw[['gazID','gazMatch_new0','gazMatch_new1','gazMatch_new2']].sort_values('gazID').groupby(['gazMatch_new0','gazMatch_new2']).first().reset_index(drop=False)
df_gpw.loc[:,'index'] = df_gpw.index
df_gpw_adhoc = df_gpw[ngazid_mask][['gazMatch_new0','gazMatch_new2','index']].merge(gazid_lookup, on=['gazMatch_new0','gazMatch_new2'], how='inner')
df_gpw_adhoc

In [ ]:
df_gpw.loc[df_gpw_adhoc['index'],['gazID','gazMatch_new2']] = df_gpw_adhoc[['gazID','gazMatch_new2']]

In [ ]:
# 68927
gazID_stats = df_gpw[df_gpw['level'] >= 2][['gazID','year']].value_counts().reset_index().groupby('gazID').agg({'year':'count', 'count': 'sum'}).reset_index().sort_values('year', ascending=False)
gazID_stats

In [ ]:
df_gpw_gazid = df_gpw[df_gpw['gazID'] != ''][['gazName','gazID','year','animal','number_heads']].groupby(['gazName','gazID','year','animal']).sum().reset_index()
df_gpw_gazid['animal'] = df_gpw_gazid['animal'].replace({'buffalo|bison': 'buffalo'})
df_gpw_gazid = df_gpw_gazid[df_gpw_gazid['animal'].isin(['cattle','goat','horse','sheep','buffalo'])]
df_gpw_gazid

In [ ]:
df_gpw_gazid['level'] = df_gpw_gazid['gazName'].str.split('.', expand=False).str.len()
df_gpw_gazid['country'] = df_gpw_gazid['gazName'].str.split('.', expand=True)[0]
df_max_level = df_gpw_gazid[['country','level','year']].value_counts().reset_index().groupby(['country']).agg({'level': 'max'}).reset_index().sort_values('country', ascending=False)

df_gpw_gazid_nodup = []

for _, rows in df_max_level.iterrows():
    df_gpw_gazid_nodup.append(
        df_gpw_gazid[
            np.logical_and(
                df_gpw_gazid['country'] == rows['country'],
                df_gpw_gazid['level'] == rows['level']
            )
        ]
    )

df_gpw_gazid_nodup = pd.concat(df_gpw_gazid_nodup)

In [ ]:
df_gpw_gazid[['country','level','year']].value_counts().reset_index().groupby(['country','level']).agg({'year':'count', 'count': 'sum'}).reset_index().sort_values('country', ascending=True)

In [ ]:
def extract_data(gazID, gaz_df_arg):
    #try:
        gaz_df = gaz_df_arg.copy()
        gaz_df.loc[:,'col'] = gaz_df['animal'] + '_' + gaz_df['year'].astype('int').astype('string')
        gaz_df.loc[:,'number_heads'] = gaz_df['number_heads'].astype('int')
        gaz_df = gaz_df.set_index('col')[['number_heads']].T.reset_index(drop=True)\
            .rename_axis(None, axis=1)#.iloc[0].to_dict()
        gaz_df['source'] = 'GPW'
        gaz_df['gazID'] = gazID
        gaz_df['gazName'] =  gaz_df_arg.iloc[0]['gazName']
        return gaz_df

In [ ]:
def extract_data(gazID, gaz_df_arg):
    #try:
        gaz_df = gaz_df_arg.copy()
        gaz_df.loc[:,'col'] = gaz_df['animal'] + '_' + gaz_df['year'].astype('int').astype('string')
        gaz_df.loc[:,'number_heads'] = gaz_df['number_heads'].astype('int')
        gaz_df = gaz_df.set_index('col')[['number_heads']].T.reset_index(drop=True)\
            .rename_axis(None, axis=1)#.iloc[0].to_dict()
        gaz_df['source'] = 'GPW'
        gaz_df['gazID'] = gazID
        gaz_df['gazName'] =  gaz_df_arg.iloc[0]['gazName']
        return gaz_df
    #except:
    #    return pd.DataFrame([])
    
args = [ (gazID, gaz_df) for gazID, gaz_df in df_gpw_gazid_nodup.groupby('gazID') ]

result = pd.concat([ df for df in parallel.job(extract_data, args, n_jobs=40) ])
result

In [ ]:
df_gpw_final = gdf_gpw[['gazID','geometry']].merge(result, on='gazID', how='inner')

ignore_cols = ['gazID','gazName','source','geometry']
cols = ignore_cols + sorted([ c for c in df_gpw_final.columns if c not in ignore_cols ])        
df_gpw_final = df_gpw_final[cols]

In [ ]:
## Dropping overlapping geometries in Australia
australia_adhoc = pd.read_csv(f'{CENSUS_DB}/australia-adhoc.csv')
df_gpw_final = df_gpw_final[np.logical_not(df_gpw_final['gazID'].isin(australia_adhoc['gazID']))]
df_gpw_final

In [ ]:
## Dropping Crimea for keep some Ukrainian data based on GLW3
df_gpw_final = df_gpw_final[np.logical_not(df_gpw_final['gazName'].str.contains('Crimea'))]

In [ ]:
gdf_gpw_countries = pd.DataFrame(df_gpw_final['gazName'].str.split('.', expand=True)[0].value_counts()).reset_index().sort_values(0).rename(columns={0:'country'})
gdf_gpw_countries[gdf_gpw_countries['country'].isin(['Mexico', 'Bolivia', 'Colombia', 'Peru', 'Paraguay', 'Chile', 'Ukraine'])]

In [ ]:
from datetime import datetime
version = datetime.today().strftime('%Y%m%d')
df_gpw_final.to_file(f'{wd}/gpw_livestock_{version}.gpkg')

In [ ]:
df_gpw_final.loc[:,'country'] = df_gpw_final['gazName'].str.split('.', expand=True)[0]
df_gpw_final.loc[:,'admin_level'] = df_gpw_final['gazName'].str.split('.', expand=False).str.len()

In [ ]:
# largest_polyon: 226098.2877692241
df_gpw_final.loc[:, 'geom_area_km'] = df_gpw_final['geometry'].to_crs('+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs').area / 1e6
largest_polygon_area = df_gpw_final[
    np.logical_and.reduce([
        ~df_gpw_final['country'].isin(['Australia', 'Canada', 'Russian Federation']),
        ~df_gpw_final['gazName'].str.contains('Alaska'),
        df_gpw_final['admin_level'] >= 3
    ])]['geom_area_km'].max()
print(f"largest_polyon: {largest_polygon_area}")

In [ ]:
df_gpw_final[df_gpw_final['geom_area_km'] == largest_polygon_area]

## GPW Ad-hoc level 2+

In [ ]:
from skmap.misc import find_files
ADHOC_DIR = f'{wd}/AdHoc_GPW'

In [ ]:
adhoc_df = pd.concat(
    [ gpd.read_file(f) for f in find_files(ADHOC_DIR, '*.gpkg')]
)
adhoc_df

In [ ]:
def _gazName(row):
    cols = ['COUNTRY', 'NAME_1', 'NAME_12', 'NAME_2', 'NAME_23', 'Region' ]
    vals =[ str(row[c]).replace(' - ','_').replace(' ','_').capitalize() for c in cols if (~pd.isnull(row[c]) and str(row[c]) != 'nan') ]
    return '.'.join(list(dict.fromkeys(vals)))

adhoc_df['gazName'] = adhoc_df.apply(_gazName, axis=1)

In [ ]:
import hashlib

cols = ['Bovins05','Bovins06','Bovins07','Bovins08','Bovins09','Bovins10','Bovins11','Bovins12','Bovins13','Bovins14','Bovins15','Bovins16','Bovins17','Bovins18','Bovins19','Bovins20','Bovins21','Bovins22','Cattle09','Cattle10','Cattle11','Cattle12','Cattle13','Cattle14']
dyear = { 'Bovins05': 2005,'Bovins06': 2006,'Bovins07': 2007,'Bovins08': 2008,'Bovins09': 2008,'Bovins10': 2010,'Bovins11': 2011,'Bovins12': 2012,'Bovins13': 2013,'Bovins14': 2014,'Bovins15': 2015,'Bovins16': 2016,'Bovins17': 2017,'Bovins18': 2018,'Bovins19': 2019,'Bovins20': 2020,'Bovins21': 2021,'Bovins22': 2022,'Cattle09': 2009,'Cattle10': 2010,'Cattle11': 2011,'Cattle12': 2012,'Cattle13': 2013,'Cattle14': 2014}

adhoc_rwise_df = []

for id, row in adhoc_df.iterrows():
    for c in cols:
        if (row[c] >= 0 and int(row[c]) != 9999):
            adhoc_rwise_df.append({
                'gazName': row['gazName'],
                'gazID': str(hashlib.md5(str(row['geometry']).encode('utf-8')).hexdigest()),
                'year': dyear[c],
                'animal': 'cattle',
                'number_heads': row[c],
                'geometry': row['geometry']
            })
    

adhoc_rwise_df = pd.DataFrame(adhoc_rwise_df)
adhoc_rwise_df['level'] = adhoc_rwise_df['gazName'].str.split('.', expand=False).str.len()
adhoc_rwise_df['country'] = adhoc_rwise_df['gazName'].str.split('.', expand=True)[0]
adhoc_rwise_df.drop_duplicates(subset=['gazID','year'], keep='last', inplace=True)
adhoc_rwise_df

In [ ]:
# Removing Guinea.Conakry & Niger.Tillabéry polygons with zero cattle
adhoc_rwise_df = adhoc_rwise_df[adhoc_rwise_df['number_heads'] > 0]

In [ ]:
args = [ (gazID, gaz_df) for gazID, gaz_df in adhoc_rwise_df.sort_values(['gazID','year']).groupby('gazID') ]

result = pd.concat([ df for df in parallel.job(extract_data, args, n_jobs=10) ])
result

In [ ]:
df_gpw_adhoc = adhoc_rwise_df.drop_duplicates(subset=['gazID','geometry'])[['gazID','geometry']].merge(result, on='gazID', how='inner')

ignore_cols = ['gazID','gazName','source','geometry']
cols = ignore_cols + sorted([ c for c in df_gpw_adhoc.columns if c not in ignore_cols ])        
df_gpw_adhoc = df_gpw_adhoc[cols]

In [ ]:
df_gpw_adhoc = gpd.GeoDataFrame(df_gpw_adhoc).set_crs(4326)

In [ ]:
from datetime import datetime
version = datetime.today().strftime('%Y%m%d')
df_gpw_adhoc.to_file(f'{wd}/gpw_livestock.adhoc_{version}.gpkg')

## FAO

In [ ]:
from skmap.misc import find_files
FAO_DIR = f'{wd}/DB_FAO'

In [ ]:
f'{wd}/DB_FAO'

In [ ]:
def _read_csv(csv_file):
    return dt.fread(csv_file).to_pandas()

args = [ (f,) for f in find_files(FAO_DIR, '*.csv') ]
fao_df = pd.concat([ df for df in parallel.job(_read_csv, args, n_jobs=16) ])
fao_df = fao_df[['YEAR','ADM_CODE','SPECIES','N0','FILE']].rename(columns={
    'YEAR': 'year',
    'ADM_CODE': 'gazID',
    'FILE': 'country_code',
    'SPECIES': 'animal',
    'N0': 'number_heads'
})

fao_df['country_code'] = fao_df['country_code'].str[:3]

In [ ]:
specie_remap = {
	'LVAL': 'All livestock',
	'ARAL': 'All ruminants',
	'BOVN': 'cattle',
	'CTAL': 'cattle',
	'CTME': 'cattle',
	'CTMI': 'cattle',
	'YKAL': 'Yaks',
	'CTDR': 'cattle',
	'BFAL': 'buffalo',
	'BFME': 'buffalo',
	'BFMI': 'buffalo',
	'BFDR': 'buffalo',
	'CAAL': 'Camelids',
	'CMAL': 'Camels',
	'ALAL': 'Alpacas',
	'LLAL': 'Llamas',
	'SRAL': 'Small ruminants',
	'SHAL': 'sheep',
	'SHMI': 'sheep',
	'GTAL': 'goat',
	'GTME': 'goat',
	'GTMI': 'goat',
	'AMAL': 'All monogastrics',
	'POAL': 'Poultry',
	'CHAL': 'Chickens',
	'CHLA': 'Layer chickens',
	'CHBO': 'Broiler chickens',
	'CHBR': 'Breeder chickens',
	'CHBY': 'Backyard chickens',
	'TUAL': 'Turkeys',
	'WAFL': 'Waterfowl',
	'DKAL': 'Ducks',
	'GSAL': 'Geese',
	'SUAL': 'Suines',
	'PGAL': 'Pigs',
	'ALEQ': 'horse', #All equines
	'EQAL': 'horse', #Equines
	'HOAL': 'horse',
	'ASAL': 'horse', #Asses
	'MUAL': 'horse', #Mules
	'UNSP': 'Unspecified',
	'WIAN': 'Wild animals',
	'ALSU': 'All susceptible'
}
fao_df['animal'] = fao_df['animal'].map(specie_remap)
fao_df = fao_df[fao_df['animal'].isin(['cattle','goat','sheep','horse','buffalo'])]
fao_df['animal'].value_counts()

In [ ]:
fao_df[fao_df['gazID'] == 2091]

In [ ]:
import fiona
from pathlib import Path

def read_geom(geom_fn):

    gdf_list = []
    for layer in fiona.listlayers(geom_fn):
        fn = Path(geom_fn).stem
        gdf = gpd.read_file(geom_fn, layer=layer )
        gdf[['file', 'layer']] = (fn, layer)
        gdf_list.append(gdf)

    return pd.concat(gdf_list)

args = [ (f,) for f in find_files(FAO_DIR, '*.gpkg') ]
gdf_fao = pd.concat([ gdf for gdf in parallel.job(read_geom, args, n_jobs=16) ])
gdf_fao

In [ ]:
gdf_fao = gdf_fao.rename(columns={'ADM_CODE': 'gazID', 'file': 'country_code'})
gdf_fao['gazID'] = gdf_fao['gazID'].astype('string')
fao_df['gazID'] = fao_df['gazID'].astype('string')

In [ ]:
# 9872
fao_dfm = fao_df.merge(gdf_fao.rename(columns={'ADM_CODE': 'gazID', 'file': 'country_code'}), on=['gazID', 'country_code'], how='inner')
fao_dfm

In [ ]:
import numpy as np
fao_dfm[np.logical_and(fao_dfm['country_code_x'] == 'ISL',fao_dfm['NAME'].isnull())]

In [ ]:
fao_dfm['NAME'].isnull().value_counts()

In [ ]:
fao_dfm[fao_dfm['NAME'].isnull()]['country_code_x'].value_counts()

In [ ]:
fao_dfm['gazName'] = fao_dfm['ADM0_NAME'] + '.' + fao_dfm['NAME']
gdf_fao = fao_dfm[['gazName', 'gazID', 'year', 'animal', 'number_heads', 'geometry', 'ADM0_NAME', 'LEVEL']].rename(columns={
    'ADM0_NAME': 'country',
    'LEVEL': 'level',
})
gdf_fao = gdf_fao[np.logical_not(fao_dfm['geometry'].isnull())]
gdf_fao['level'] = gdf_fao['level'].astype('int') + 1
gdf_fao = gdf_fao[gdf_fao['year'] >= 2000]
gdf_fao.drop_duplicates(subset=['gazID','year'], keep='last', inplace=True)
gdf_fao['number_heads'] = gdf_fao['number_heads'].astype('float')
gdf_fao = gdf_fao[np.logical_not(np.isnan(gdf_fao['number_heads']))]
gdf_fao = gdf_fao[gdf_fao['number_heads'] > 0]
gdf_fao

In [ ]:
args = [ (gazID, gaz_df) for gazID, gaz_df in gdf_fao.sort_values(['gazID','year']).groupby('gazID') ]

result = pd.concat([ df for df in parallel.job(extract_data, args, n_jobs=10) ])

In [ ]:
gdf_fao_agg = gdf_fao.drop_duplicates(subset=['gazID','geometry'])[['gazID','geometry']].merge(result, on='gazID', how='inner')

ignore_cols = ['gazID','gazName','source','geometry']
cols = ignore_cols + sorted([ c for c in gdf_fao_agg.columns if c not in ignore_cols ])        
gdf_fao_agg = gdf_fao_agg[cols]
gdf_fao_agg = gpd.GeoDataFrame(gdf_fao_agg, geometry=gpd.GeoSeries(gdf_fao_agg['geometry']))
gdf_fao_agg['source'] = 'FAO'
gdf_fao_agg

In [ ]:
from datetime import datetime

version = datetime.today().strftime('%Y%m%d')
gdf_fao_agg.to_file(f'{wd}/gpw_livestock.fao_{version}.gpkg')

## Malek et al., 2024

In [ ]:
from skmap.misc import find_files
MALEK_DIR = f'{wd}/Malek_2024/EU_2020'

In [ ]:
import fiona
from pathlib import Path

def read_geom(geom_fn):

    gdf_list = []
    for layer in fiona.listlayers(geom_fn):
        fn = Path(geom_fn).stem
        gdf = gpd.read_file(geom_fn, layer=layer ).to_crs(4326)
        gdf[['file', 'layer']] = (fn, layer)
        gdf_list.append(gdf)

    return pd.concat(gdf_list)

args = [ (f,) for f in find_files(MALEK_DIR, '*.gpkg') ]
gdf = pd.concat([ df for df in parallel.job(read_geom, args, n_jobs=16) ])
gdf

In [ ]:
gdf.columns

In [ ]:
import hashlib

cols = ['buffalo','catt_tot','cattle tot','cattle','cattle2000','cattle2010','cattle2020','cattle_tot','Cattle_tot','goat','goat_t','goats','sheep','sheep2000','sheep2010','sheep2020','sheep_t','sheep_tot','Sheep_tota']
dyear = { 'buffalo': 2020, 'catt_tot': 2020, 'cattle tot': 2020, 'cattle': 2020, 'cattle2000': 2000, 'cattle2010': 2010, 'cattle2020': 2020, 'cattle_tot': 2020, 'Cattle_tot': 2020, 'goat': 2020, 'goat_t': 2020, 'goats': 2020, 'sheep': 2020, 'sheep2000': 2000, 'sheep2010': 2010, 'sheep2020': 2020, 'sheep_t': 2020, 'sheep_tot': 2020, 'Sheep_tota': 2020 }
danimal = {'buffalo': 'buffalo','catt_tot': 'cattle','cattle tot': 'cattle','cattle': 'cattle','cattle2000': 'cattle','cattle2010': 'cattle','cattle2020': 'cattle','cattle_tot': 'cattle','Cattle_tot': 'cattle','goat': 'goat','goat_t': 'goat','goats': 'goat','sheep': 'sheep','sheep2000': 'sheep','sheep2010': 'sheep','sheep2020': 'sheep','sheep_t': 'sheep','sheep_tot': 'sheep','Sheep_tota': 'sheep'}

malek_rwise_df = []

for id, row in gdf.iterrows():
    for c in cols:
        if (row[c] >= 0 and int(row[c]) != 9999):
            malek_rwise_df.append({
                'gazName': row['local'],
                'gazID': str(hashlib.md5(str(row['geometry']).encode('utf-8')).hexdigest()),
                'year': dyear[c],
                'animal': danimal[c],
                'number_heads': row[c],
                'country': str(row['country']),
                'geometry': row['geometry']
            })
    

malek_rwise_df = pd.DataFrame(malek_rwise_df)
malek_rwise_df['level'] = malek_rwise_df['gazName'].str.split('.', expand=False).str.len()
malek_rwise_df.drop_duplicates(subset=['gazID','year'], keep='last', inplace=True)
malek_rwise_df

In [ ]:
malek_rwise_df[malek_rwise_df['number_heads'] == 0]['country'].value_counts()

In [ ]:
malek_rwise_df = malek_rwise_df[malek_rwise_df['number_heads'] > 0]
malek_rwise_df

In [ ]:
args = [ (gazID, gaz_df) for gazID, gaz_df in malek_rwise_df.sort_values(['gazID','year']).groupby('gazID') ]

result = pd.concat([ df for df in parallel.job(extract_data, args, n_jobs=10) ])
result

In [ ]:
malek_rwise = malek_rwise_df.drop_duplicates(subset=['gazID','geometry'])[['gazID','geometry']].merge(result, on='gazID', how='inner')

ignore_cols = ['gazID','gazName','source','geometry']
cols = ignore_cols + sorted([ c for c in malek_rwise.columns if c not in ignore_cols ])        
malek_rwise = malek_rwise[cols]
malek_rwise = gpd.GeoDataFrame(malek_rwise, geometry=gpd.GeoSeries(malek_rwise['geometry']))
malek_rwise['source'] = 'Malek et al., 2024'
malek_rwise

In [ ]:
from datetime import datetime

version = datetime.today().strftime('%Y%m%d')
malek_rwise.to_file(f'{wd}/gpw_livestock.malek.2024_{version}.gpkg')

## FAO Level 1

In [ ]:
adb_meta = f'{CENSUS_DB}/_meta'
meta_dataseries = pd.read_csv(f'{adb_meta}/inv_dataseries.csv')
meta_table = pd.read_csv(f'{adb_meta}/inv_tables.csv')
metadata = meta_table.merge(meta_dataseries, on='datID')

In [ ]:
FAOSAT_tab_id = metadata[metadata['name'] == 'faostat']['tabID'].iloc[0]
FAOSAT_tab_id

In [ ]:
df_gpw_l1 = df.query(f'year >= 2000 & number_heads >= 0 & tabID == {FAOSAT_tab_id}')
df_gpw_l1 = df_gpw_l1[df_gpw_l1['ontoMatch'].str.contains('close--')]

df_gpw_l1['animal'] = df_gpw_l1['animal'].replace({'buffalo|bison': 'buffalo'})
df_gpw_l1 = df_gpw_l1[df_gpw_l1['animal'].isin(['cattle','goat','horse','sheep','buffalo'])]
df_gpw_l1

In [ ]:
# pip install pyogrio
def read_geom(geom_dir, country, level):
    try:
        gdf = gpd.read_file(f'{geom_dir}/{country}.gpkg', layer=f'al{level}' )
        return gdf[['gazID', 'gazName','gazClass','geoID','geometry']]
    except:
        try:
            gdf = gpd.read_file(f'{geom_dir}/{country}.gpkg', layer=f'ADM{int(level) - 1}' )
            return gdf[['gazID', 'gazName','gazClass','geoID','geometry']]
        except:
            return gpd.GeoDataFrame([])
    
args = [ (adb_geometries_path, cl.split('#')[0], cl.split('#')[1]) for cl in pd.unique(df_gpw_l1['country']+ '#' + df_gpw_l1['level'].astype(str)) ]
gdf_gpw_l1 = pd.concat([ df for df in parallel.job(read_geom, args, n_jobs=40) ])

# Only geometries with livesotck information 
gdf_gpw_l1 = gdf_gpw_l1[gdf_gpw_l1['gazName'].isin(df_gpw_l1['gazName'].unique())]
gdf_gpw_l1 = gdf_gpw_l1.drop_duplicates(['gazID'], keep='first', ignore_index=True)
gdf_gpw_l1

In [ ]:
gdf_gpw_l1['geom_area_km'] = gdf_gpw_l1['geometry'].to_crs('+proj=igh +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs').area / 1e6

In [ ]:
result_l1 = []
for gazID in df_gpw_l1['gazID'].unique():
    mask = (df_gpw_l1['gazID'] == gazID)
    gaz_df = df_gpw_l1[mask].copy()
    gaz_df.loc[:,'col'] = gaz_df['animal'] + '_' + gaz_df['year'].astype('int').astype('string')
    gaz_df.loc[:,'number_heads'] = gaz_df['number_heads'].astype('int')
    gaz_df = gaz_df.set_index('col')[['number_heads']].T.reset_index(drop=True)\
        .rename_axis(None, axis=1).iloc[0].to_dict()
    gaz_df['source'] = 'FAOSTAT'
    gaz_df['gazID'] = gazID
    gaz_df['gazName'] =  df_gpw_l1[mask].iloc[0]['gazName']
    result_l1.append(gaz_df)

result_l1 = pd.DataFrame(result_l1)
result_l1

In [ ]:
#df_alaska = df[df['gazName'].str.contains('Alaska')].query(f"year >= 2000 & number_heads >= 0 & animal in ('cattle','goat','horse','sheep','buffalo')" )
#df_alaska['country'] = df_alaska['gazName'].str.split('.', expand=True)[0] + '.' + df_alaska['gazName'].str.split('.', expand=True)[1]
#df_alaska

In [ ]:
import joblib
gdf_gpw_l1_final = gdf_gpw_l1[['gazID','geometry']].merge(result_l1, on='gazID', how='inner')

ignore_cols = ['gazID','gazName','source','geometry']
cols = ignore_cols + sorted([ c for c in gdf_gpw_l1_final.columns if c not in ignore_cols ])        
gdf_gpw_l1_final = gdf_gpw_l1_final[cols]

gdf_gpw_l1_final.to_file('faostat_livestock_all.gpkg')
joblib.dump(gdf_gpw_l1_final, 'faostat_livestock_all.lz4', compress='lz4')
gdf_gpw_l1_final

In [ ]:
#largest_polygon_area = 226,098
gdf_gpw_l1_final = gdf_gpw_l1[gdf_gpw_l1['geom_area_km'] <= largest_polygon_area][['gazID','geometry']].merge(result_l1, on='gazID', how='inner')

ignore_cols = ['gazID','gazName','source','geometry']
cols = ignore_cols + sorted([ c for c in gdf_gpw_l1_final.columns if c not in ignore_cols ])        
gdf_gpw_l1_final = gdf_gpw_l1_final[cols]

from datetime import datetime
version = datetime.today().strftime('%Y%m%d')
gdf_gpw_l1_final.to_file(F'{ard_census_dir}/faostat_livestock.adhoc_{version}.gpkg')

#gdf_gpw_l1_final.to_file('faostat_livestock.gpkg')
#gdf_gpw_l1_final

## GLW3

In [ ]:
from eumap.misc import find_files
from eumap.mapper import SpaceOverlay
import geopandas as gpd
import pandas as pd

In [ ]:
raster_dir = '/mnt/tupi/WRI/livestock_global_modeling/Datasets/GLW3/'
vector_adm_fn = '/mnt/tupi/WRI/livestock_global_modeling/Datasets/gaul_2015_2014_2.gpkg'
glw_version = 'glw3'
raster_files = find_files(raster_dir, '*Aw.tif') + find_files(raster_dir + '/cattle', '*Areakm*') + find_files(raster_dir, '*Ps.tif')

In [ ]:
country_code_fn = '/mnt/tupi/WRI/livestock_global_modeling/Datasets/gadm36_country_codes.csv'
country_code = pd.read_csv(country_code_fn)
country_code

In [ ]:
vector_adm = gpd.read_file(vector_adm_fn)
vector_adm['ADM0_NAME'] = vector_adm['ADM0_NAME'].replace({
    'United States of America': 'United States',
    'Viet Nam': 'Vietnam',
    'Iran  (Islamic Republic of)': 'Iran',
    'Dem People\'s Rep of Korea': 'North Korea',
    'United Republic of Tanzania': 'Tanzania',
    'Lao People\'s Democratic Republic': 'Laos',
    'The former Yugoslav Republic of Macedonia': 'Macedonia',
    'Russian Federation': 'Russia',
    'Congo': 'Republic of Congo',
    'U.K. of Great Britain and Northern Ireland': 'United Kingdom',
    'Syrian Arab Republic': 'Syria',
    'Moldova, Republic of': 'Moldova',
    'Czech Republic': 'Czechia'
})

In [ ]:
vector_adm = vector_adm.merge(country_code, on='ADM0_NAME', how='left')
vector_adm

In [ ]:
vector_adm_pts = vector_adm.representative_point()
spc_overlay = SpaceOverlay(points=gpd.GeoDataFrame(geometry=vector_adm_pts), fn_layers=raster_files)
result = spc_overlay.run()
result = result[result.columns[2:]]
result = result.set_axis([ f'{glw_version}_pixel_{c[2:]}'  for c  in result.columns ], axis=1)
print(result.shape)

In [ ]:
vector_adm

In [ ]:
vector_adm_overlaid = pd.merge(vector_adm, result,  left_index=True, right_index=True)

In [ ]:
vector_adm_overlaid['polygon_area_km'] = vector_adm_overlaid.to_crs('+proj=igh +towgs84=0,0,0').area / 1000000
vector_adm_overlaid.columns

In [ ]:
from pathlib import Path 
import pandas as pd
aw_columns = vector_adm_overlaid.columns[vector_adm_overlaid.columns.str.contains('Aw')]
metadata_dir = '/mnt/tupi/WRI/livestock_global_modeling/Datasets/GLW3'

for aw_col in aw_columns:
    
    print(f"Calculating vector_{aw_col}")
    
    atype = aw_col.split('_')[2]
    metadata_fn = Path(metadata_dir).joinpath(f'1_{atype}_2010_Metadata.html')
    print(metadata_fn)
    metadata = pd.read_html(metadata_fn)[0].rename(columns={'Unnamed: 0':'GID_0', 'Country': 'Country_name'})
    metadata['faostat_census_ratio'] = metadata['FAOSTAT'] / metadata['Census']
    metadata = metadata[['GID_0', 'Year', 'Source', 'faostat_census_ratio']].add_suffix(f'_{atype}').add_prefix(f'{glw_version}_mtd_')
    
    right_on = f'{glw_version}_mtd_GID_0_{atype}'
    vector_adm_overlaid = pd.merge(vector_adm_overlaid, metadata, how='left', left_on=f'GID_0', right_on=right_on).drop(columns=[right_on])
        
    vector_adm_overlaid[f'polygon_{atype}'] = (vector_adm_overlaid['polygon_area_km'] / vector_adm_overlaid[f'{glw_version}_pixel_Areakm']) * vector_adm_overlaid[f'{glw_version}_pixel_{atype}_2010_Aw']
    vector_adm_overlaid[f'polygon_{atype}'] = vector_adm_overlaid[f'polygon_{atype}'] / pd.to_numeric(vector_adm_overlaid[f'{glw_version}_mtd_faostat_census_ratio_{atype}'])
    
    print(vector_adm_overlaid.shape)

In [ ]:
vector_adm_overlaid.columns = map(str.lower, vector_adm_overlaid.columns)
columns_sorted = sorted(list(vector_adm_overlaid.columns))
columns_sorted.remove('geometry')
columns_sorted

In [ ]:
import numpy as np
cats = [ 'bf', 'ch', 'ct', 'dk', 'gt', 'ho', 'pg', 'sh' ]
for c in cats:
    mask = np.logical_or((vector_adm_overlaid[f'glw3_pixel_{c}_2010_ps'] == 1),~np.isfinite(vector_adm_overlaid[f'glw3_pixel_{c}_2010_aw']))
    
    print(f"Removing interpolated samples for {c} {np.sum(mask.astype('int'))}")
    vector_adm_overlaid.loc[mask, f'glw3_pixel_{c}_2010_aw'] = np.nan
    vector_adm_overlaid.loc[mask, f'polygon_{c}'] = np.nan
    #vector_adm_overlaid[f'glw3_pixel_{c}_2010_ps'].value_counts()

In [ ]:
cats = [ 'bf', 'ch', 'ct', 'dk', 'gt', 'ho', 'pg', 'sh' ]
for c in cats:
    mask = (vector_adm_overlaid[f'glw3_pixel_{c}_2010_ps'] == 1)
    vector_adm_overlaid[f'polygon_{c}'] = vector_adm_overlaid[f'polygon_{c}'].round(0)

In [ ]:
vector_adm_overlaid[np.logical_and(vector_adm_overlaid['adm0_name'] == 'Brazil',vector_adm_overlaid['adm2_name'] == 'Querencia')]

In [ ]:
# IBGE: https://sidra.ibge.gov.br/tabela/3939#resultado

import numpy as np
vector_adm_overlaid[np.logical_and(vector_adm_overlaid['adm2_name'] == 'Querencia', vector_adm_overlaid['adm0_name'] == 'Brazil')][[
    'polygon_ct',f'{glw_version}_mtd_year_ct',f'{glw_version}_mtd_faostat_census_ratio_ct',
    'polygon_area_km', f'{glw_version}_pixel_areakm', f'{glw_version}_pixel_ct_2010_aw'
]]

In [ ]:
vector_adm_overlaid = vector_adm_overlaid.query(' | '.join([ f'polygon_{c} >= 0' for c in cats ]))
vector_adm_overlaid

In [ ]:
vector_adm_overlaid

In [ ]:
vector_adm_overlaid.loc[:,'gazName'] = vector_adm_overlaid['adm0_name'] + '.' + vector_adm_overlaid['adm1_name'] + '.' + vector_adm_overlaid['adm2_name']
vector_adm_overlaid.loc[:,'index'] = vector_adm_overlaid.index

cat_map = {    
    'ct': 'cattle',
    'gt': 'goat',
    'ho': 'horse',
    'sh': 'sheep',
    'bf': 'buffalo'
}

result = []

for c in list(cat_map.keys()):
    col_y = f'glw3_mtd_year_{c}'
    col_c = f'polygon_{c}'
    for year in vector_adm_overlaid[col_y].unique():
        if year >= 2000:
            gaz_df = vector_adm_overlaid[np.logical_and(vector_adm_overlaid[col_y] == year, vector_adm_overlaid[col_c] > 0)]
            rename_cols = {}
            rename_cols[col_c] = f'{cat_map[c]}_{int(year)}'
            gaz_df = gaz_df[['index',col_c]].rename(columns=rename_cols)
            result.append(gaz_df)
            print(col_c, gaz_df.shape)

result = pd.concat(result)
result

In [ ]:
ignore_cols = ['index']
cols = ignore_cols + sorted([ c for c in result.columns if c not in ignore_cols ])        
result = result[cols].fillna(0).groupby(['index']).sum().reset_index()
result.loc[:,cols[1:]] = result[cols[1:]].replace({0:np.nan})

In [ ]:
result = vector_adm_overlaid[['index','gazName','geometry']].merge(result, on='index')
result['gazID'] = 'NA'
result['source'] = 'fao_glw3'

In [ ]:
#df_gpw_final = gdf_gpw[['gazID','geometry']].merge(result, on='gazID', how='inner')

ignore_cols = ['gazID','gazName','source','geometry']
cols = ignore_cols + sorted([ c for c in result.columns if c not in ignore_cols ])        
df_glw = result[cols]

In [ ]:
df_glw

In [ ]:
df_glw.to_file('fao_glw3_livestock.gpkg')

In [ ]:
df_glw

## Census data integration

In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
#ard_dir = '/mnt/tupi/WRI/livestock_global_modeling/livestock_census_ard'
gpw_df = gpd.read_file(f'{wd}/gpw_livestock_20250201.gpkg')
gpw_ah_df = gpd.read_file(f'{wd}/gpw_livestock.adhoc_20250129.gpkg')
fao_df = gpd.read_file(f'{wd}/gpw_livestock.fao_20250201.gpkg')
malek_df = gpd.read_file(f'{wd}/gpw_livestock.malek.2024_20250129.gpkg')

faostat_df = gpd.read_file(f'{wd}/faostat_livestock.adhoc_20250227.gpkg')
#faoglw3_df = gpd.read_file(f'{wd}/fao_glw3_livestock.gpkg')

In [ ]:
gpw_df = pd.concat([gpw_df, gpw_ah_df.to_crs(4326), fao_df.to_crs(4326), malek_df.to_crs(4326)])

In [ ]:
gpw_df['level'] = gpw_df['gazName'].str.split('.', expand=False).str.len()
gpw_df['country'] = gpw_df['gazName'].str.split('.', expand=True)[0]

faostat_df['level'] = faostat_df['gazName'].str.split('.', expand=False).str.len()
faostat_df['country'] = faostat_df['gazName'].str.split('.', expand=True)[0]

In [ ]:
#faoglw3_df['level'] = faoglw3_df['gazName'].str.split('.', expand=False).str.len()
#faoglw3_df['country'] = faoglw3_df['gazName'].str.split('.', expand=True)[0]

#faoglw3_df['country'] = faoglw3_df['country'].replace({
#    'United States': 'United States of America',
#    'Russia': 'Russian Federation',
#    'Netherlands': 'Netherlands (Kingdom of the)',
#    'Turkey': 'Türkiye',
#    'Czech Republic': 'Czechia',
#    'Macedonia': 'North Macedonia',
#    'Moldova': 'Republic of Moldova',
#    'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
#    'Burkina Faso': 'Burkina_faso',
#    'Syria': 'Syrian Arab Republic',
#    'Swaziland': 'Eswatini'
#})

#gpw_df_point = gpw_df.copy()
#gpw_df_point.geometry = gpw_df.representative_point()

#r_intesects = gpd.sjoin(faoglw3_df, luckinet_df, how='inner')
#r_whitin = gpd.sjoin(faoglw3_df, gpw_df_point, how='inner')
#faoglw3_df_new = faoglw3_df.drop(index=list(r_whitin.index.unique()))

#faoglw3_df_new = faoglw3_df.drop(index=faoglw3_df[
#    faoglw3_df['country'].isin(
#        list(gpw_df['country'].unique()) + list(faostat_df_new['country'].unique())
#    )
#].index)

In [ ]:
faostat_df_new = faostat_df.drop(index=faostat_df[faostat_df['country'].isin(list(gpw_df['country'].unique()))].index)

In [ ]:
print(faostat_df_new.shape, faostat_df.shape)

In [ ]:
gpw_final = pd.concat([gpw_df, faostat_df_new])
gpw_final

In [ ]:
import numpy as np
animals = ['cattle','goat','horse','sheep']
for c in animals:
    animal_cols = gpw_final.columns[gpw_final.columns.str.contains(c)]
    gpw_final.loc[:,f'{c}_nyears'] = np.sum(( np.logical_not(np.isnan(gpw_final[animal_cols]).to_numpy()) ).astype('int'), axis=1)
gpw_final = gpw_final.reset_index(drop=True)
gpw_final

In [ ]:
to_remove = gpw_final[
    np.logical_and(
        gpw_final['source'] == 'FAO',
        np.logical_or.reduce([
            gpw_final['gazName'].str.contains('Brazil'),
            gpw_final['gazName'].str.contains('Argentina'),
            gpw_final['gazName'].str.contains('Russia')
        ])
    )].index
print(f"Removing {len(to_remove)} FAO samples")

In [ ]:
gpw_final = gpw_final.drop(to_remove)
gpw_final

In [ ]:
import joblib
joblib.dump(gpw_final, f'{wd}/gpw_livestock.animals_gpw.fao.glw3_polygon.samples_20000101_20231231_go_epsg.4326_v1.lz4', compress='lz4')
gpw_final.to_file(f'{wd}/gpw_livestock.animals_gpw.fao.faostat.malek.2024_polygon.samples_20000101_20231231_go_epsg.4326_v1.gpkg')